# Getting the Response From ESPN API and Converting it into a JSON file

In [1]:
# Import a library that allows to make HTTP request
import requests
import json
# Set the API endpoint
url = "https://hs-consumer-api.espncricinfo.com/v1/pages/matches/current?lang=en&latest=true"

# Use the library to perform an HTTP GET request to the URL
response = requests.get(url)
response_dict = json.loads(response.text)

# Function to clear Html tags


In [2]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

# Generating Summary of Ongoing Matches

In [3]:
summary_msg = []
summary_msg.append("---CURRENT GAMES IN LIST---")
for match_dict in response_dict['matches']:
    match_details = match_dict['slug'].upper()
    match_status = match_dict['statusText']
    summary_msg.append("---")
    summary_msg.append(match_details)
    summary_msg.append(match_status)
    try:
        teams={}
        toss ={}
        toss.update({1 : "Bat"})
        toss.update({2 : "Bowl"})
        for team in match_dict['teams']:
            team_name = team['team']['name']
            team_id = team['team']['id']
            teams.update({team_id : team_name})
        tossId = match_dict['tossWinnerTeamId']
        fieldId = match_dict['tossWinnerChoice']
        toss_status = "{} have won the toss and decided to {} first".format(teams[tossId],toss[fieldId])
        summary_msg.append(toss_status)
    except:
        pass
    summary_msg.append("---")


In [4]:
for msg in summary_msg:
    print(msg)

---CURRENT GAMES IN LIST---
---
AUSTRALIA-VS-ENGLAND-2ND-TEST
Day 5 - Session 2: England need 290 runs.
Australia have won the toss and decided to Bat first
---
---
BALOCHISTAN-VS-NORTHERN-PAKISTAN-28TH-MATCH
Northern lead by 144 runs.
Northern have won the toss and decided to Bowl first
---
---
SINDH-VS-SOUTHERN-PUNJAB-PAKISTAN-29TH-MATCH
Southern PNJ lead by 25 runs.
Southern PNJ have won the toss and decided to Bowl first
---
---
KHYBER-PAKHTUNKHWA-VS-CENTRAL-PUNJAB-PAKISTAN-30TH-MATCH
Khyber lead by 334 runs.
Central PNJB have won the toss and decided to Bowl first
---
---
WELLINGTON-WOMEN-VS-CANTERBURY-WOMEN-9TH-MATCH
Well Women won by 45 runs
Well Women have won the toss and decided to Bat first
---
---
TASMANIA-WOMEN-VS-QUEENSLAND-WOMEN-4TH-MATCH
Tas Women won by 5 wickets (with 5 balls remaining) (D/L method)
Tas Women have won the toss and decided to Bowl first
---
---
VICTORIA-WOMEN-VS-NEW-SOUTH-WALES-WOMEN-3RD-MATCH
NSW Women won by 26 runs (D/L method)
VIC Women have won th

# Getting The IDs of Current Games

In [5]:
game_ids = []
for match_dict in response_dict['matches']:
    if match_dict['coverage']=="Y":
        series_id = match_dict['series']['objectId']
        match_id = match_dict['objectId']
        game_ids.append((series_id,match_id))

In [6]:
game_ids

[(1263452, 1263463),
 (1289602, 1289614),
 (1293040, 1293043),
 (1293683, 1293716),
 (1280193, 1280400),
 (1293683, 1293717),
 (1269637, 1269651),
 (1291193, 1291215),
 (1291193, 1291216),
 (1289602, 1289604),
 (1293683, 1293763),
 (1293683, 1293764),
 (1269637, 1269652),
 (1279953, 1279965),
 (1293683, 1293889),
 (1293683, 1293890),
 (1280193, 1280401),
 (1293683, 1293891),
 (1269637, 1269653),
 (1291193, 1291217)]

# Generating Messages for the top 3 games

In [7]:
game_msgs = []
game_msgs.append(["-----------------------------------------------------------------------------------------"])
game_msgs.append(["-------------------------------------NEW UPDATES ----------------------------------------"])
for game_id in game_ids[:3]:
    final_msgs=[]
    final_msgs.append("--------------------------------------------------")
    # Set the API endpoint
    url = "https://hs-consumer-api.espncricinfo.com/v1/pages/match/details?lang=en&seriesId={}&matchId={}&latest=true".format(game_id[0],game_id[1])

    # Use the library to perform an HTTP GET request to the URL
    response = requests.get(url)
    response_dict = json.loads(response.text)
    
    # Generating Initial Message
    match_details = response_dict['match']['slug'].upper()
    match_status = response_dict['match']['statusText']
    final_msgs.append(match_details)
    final_msgs.append(match_status)
    final_msgs.append("Status: " + response_dict['match']['status'])
    # Getting the current score
   
    curr_score = response_dict['match']['teams'][0]['score']
    country = response_dict['match']['teams'][0]['team']['name']
    country_2 = response_dict['match']['teams'][1]['team']['name']
    curr_score_2 = response_dict['match']['teams'][1]['score']
    msg_1 = "{}'s score is: {}".format(country, curr_score)
    msg_2 = "{}'s score is: {}".format(country_2, curr_score_2)
    msg_1 = msg_1.replace("&","and")
    msg_2 = msg_2.replace("&","and")
    final_msgs.append(msg_1)  
    final_msgs.append(msg_2)
    # Getting the recent commentaries
    i = 0
    try:
        for comments in response_dict['recentBallCommentary']['ballComments']:
            if comments['isFour'] or comments['isSix'] or comments['isWicket'] or i==0:
                overs = comments['oversActual']
                overnum = comments['overNumber']
                ballnum = comments['ballNumber']
                innings = comments['inningNumber']
                msg_2 = "Innings no:{} and {} overs have been bowled".format(innings, overs)
                final_msgs.append(comments['title'])
                if i == 0:
                    final_msgs.append(msg_2)
                if comments['isFour']:
                    final_msgs.append("Its a Boundary --4--")
                if comments['isSix']:
                    final_msgs.append("Its a maximum --6--")
                if comments['isWicket']:
                    final_msgs.append("Wicket down!")
                    final_msgs.append(comments['dismissalText']['commentary'])
                
                try:
                    final_msgs.append(comments['commentTextItems'][0]['html'])
                except:
                    pass
                if(i==0):
                    final_msgs.append("        RECENT HIGHLIGHTS         ")
                i+=1
        runs = ""
        for comments in response_dict['recentBallCommentary']['ballComments']:
            runs += str(comments['totalRuns']) + " "
            if comments['ballNumber'] == 6:
                runs += "|" + " "
        final_msgs.append(runs)
        
        for comments in response_dict['recentBallCommentary']['ballComments']:
            if comments['over'] != None:
                batsman_1 = comments['over']['overEndBatsmen'][0]['player']['longName']
                batsman_2 = comments['over']['overEndBatsmen'][1]['player']['longName']
                bowler_1 = comments['over']['overEndBowlers'][0]['player']['longName']
                bowler_2 = comments['over']['overEndBowlers'][1]['player']['longName']
                batsmen = "Batsmen in crease: {} and {}".format(batsman_1,batsman_2)
                bowlers = "Current Bowlers: {} and {}".format(bowler_1, bowler_2)
                final_msgs.append(batsmen)
                final_msgs.append(bowlers)
                break
                
    except:
        pass
    # Clearing HTML
    for i in range(len(final_msgs)):
        final_msgs[i] = cleanhtml(final_msgs[i])
    final_msgs.append("-------------------------------------------------")
    
    game_msgs.append(final_msgs)

In [8]:
for game in game_msgs:
    for msg in game:
        print(msg)


-----------------------------------------------------------------------------------------
-------------------------------------NEW UPDATES ----------------------------------------
--------------------------------------------------
AUSTRALIA-VS-ENGLAND-2ND-TEST
Day 5 - Session 2: England need 290 runs.
Status: Live
Australia's score is: 473/9d and 230/9d
England's score is: 236 and 178/7
Green to Buttler
Innings no:4 and 101.6 overs have been bowled
beaten again!
        RECENT HIGHLIGHTS         
0 0 | 0 1 0 0 0 0 | 0 0 0 0 0 0 | 0 0 0 0 0 0 | 
Batsmen in crease: Ollie Robinson and Jos Buttler
Current Bowlers: Nathan Lyon and Cameron Green
-------------------------------------------------
--------------------------------------------------
WELLINGTON-VS-CANTERBURY-9TH-MATCH
Canterbury won by 6 wickets (with 21 balls remaining)
Status: RESULT
Wellington's score is: 151/9
Canterbury's score is: 157/4
Sears to Fletcher
Innings no:2 and 16.2 overs have been bowled
        RECENT HIGHLIGHTS 

In [9]:
import requests
import numpy as np
import pandas as pd
import html

# Bot Handling - Getting the Chat IDs from latest Requests

In [10]:
bot_url = "https://api.telegram.org/bot5019574567:AAGhEiWD4ZslxOBBNVI9-NUng8lmpeCiOnY/getUpdates"
response = requests.get(bot_url)
response_dict = json.loads(response.text)
chat_ids = []
latest_id = pd.read_csv("./lastId.csv").values[0][0]
for update_dict in response_dict['result']:
    try:
        if (update_dict['message']['text'] == "/command1" and update_dict['update_id'] > latest_id):
            chat_id = update_dict['message']['chat']['id']
            chat_ids.append(chat_id)
        last_id = update_dict['update_id']
    except:
        pass
last_id = pd.DataFrame(np.array([last_id]))
last_id.to_csv("lastId.csv", index=False)
chat_ids = np.unique(np.array(chat_ids))
chat_ids = chat_ids.tolist()
print(chat_ids)

[5029564645]


# Sending The Messages to Users using Telegram API

In [11]:

def telegram_bot_sendText(chat_ids, text_message):
    bot_token = '5019574567:AAGhEiWD4ZslxOBBNVI9-NUng8lmpeCiOnY'
    responses = []
    for chat_id in chat_ids:
        bot_chatid = str(chat_id)
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatid + \
                 '    &parse_mode=HTML&text=' + text_message
        response = requests.get(send_text)
        responses.append(response.json())
    return responses

In [12]:
for game in game_msgs:
    for msg in game:
        telegram_bot_sendText(chat_ids, msg)